# MBPP Replicate

In [1]:
from human_eval.data import write_jsonl, read_problems, MBPP_TEST
from human_eval.evaluation import evaluate_functional_correctness
import logging
import sys
import time, datetime
from langchain_community.llms import Replicate

%load_ext autoreload
%autoreload 2

In [2]:
def remove_triple_backticks_at_edges(text: str) -> str:
    """
    Removes triple backticks (` ``` `) if they appear at the beginning or the end of the text.

    :param text: The input text as a string.
    :return: A string with triple backticks removed from the edges.
    """
    s1 = "```python\n"
    s2 = "```python"
    s3 = "```"
    for s in [s1, s2, s3]:
        if text.startswith(s):
            text = text[len(s):].lstrip()      # Remove a version of backticks + extra whitespace at the start
    if text.endswith(s3):
        text = text[: -len(s3)].rstrip()       # Remove the backticks + extra whitespace at the end
    return text


def remove_last_triple_backticks(text: str) -> str:
    """
    Removes everything starting from the last triple backticks (` ``` `) to the end.

    :param text: The input text as a string.
    :return: A string with all text removed after the last set of triple backticks
    """
    s = "```"
    idx = text.rfind( s )
    return text[:idx]

In [3]:
tasks = read_problems(MBPP_TEST)
print(f'Type of tasks: {type(tasks)}\n')

counter = 0
for k,v in tasks.items():
    if counter == 10:
        break
    print(f'task_id: {k}', type(k))
    for k2, v2 in v.items():
        print(f'\n{k2}:\n{v2}')
    print('\n' + '='*100 + '\n')
    counter += 1

Type of tasks: <class 'dict'>

task_id: 11 <class 'str'>

task_id:
11

test:

assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H"

prompt:
Write a python function to remove first and last occurrence of a given character from the string.

canonical_solution:
def remove_Occ(s,ch): 
    for i in range(len(s)): 
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    for i in range(len(s) - 1,-1,-1):  
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    return s 

entry_point:
None


task_id: 12 <class 'str'>

task_id:
12

test:

assert sort_matrix([[1, 2, 3], [2, 4, 5], [1, 1, 1]])==[[1, 1, 1], [1, 2, 3], [2, 4, 5]]
assert sort_matrix([[1, 2, 3], [-2, 4, -5], [1, -1, 1]])==[[-2, 4, -5], [1, -1, 1], [1, 2, 3]]
assert sort_matrix([[5,8,9],[6,4,3],[2,1,4]])==[[2, 1, 4], [6, 4, 3], [5, 8, 9]]

prompt:
Write a function to sort a given matrix in ascending order accordi

In [11]:
llama3_8b = Replicate(
    model        = 'meta/meta-llama-3-8b-instruct:5a6809ca',
    model_kwargs = {"temperature": 0.95,"top_p": 1, "max_new_tokens":512}
)

solar_10b = Replicate(
    model        = 'nateraw/nous-hermes-2-solar-10.7b:1e918ab6ffd5872c21fba21a511f344fd12ac0edff6302c9cd260395c7707ff4',
    model_kwargs = {"temperature": 0.95,"top_p": 1, "max_new_tokens":512}
)

qwen = Replicate(
    model        = 'lucataco/qwen1.5-7b:f85bec5b21ba0860e0f200be6ef5af9d5a65b974b9f99e36eb036d21eab884de',
    model_kwargs = {"temperature": 0.95,"top_p": 1, "max_new_tokens":512}
)

phixtral_7b = Replicate(
    model        = 'lucataco/phixtral-2x2_8:25d7b93bb0ec9e8dd94fcc69adc786759243a5628ba5574bd9609d6abafe57cf',
    model_kwargs = {"temperature": 0.95,"top_p": 1, "max_new_tokens":512}
)

mamba = Replicate(
    model        = 'adirik/mamba-2.8b:864ed9237f28ff2e8eb55ffc07e9f0d7857888cacb9c32a92081f24f49a861db',
    model_kwargs = {"temperature": 0.95,"top_p": 1, "max_new_tokens":512}
)

mamba2 = Replicate(
    model        = 'adirik/mamba-2.8b:571abd73203a3dd3d7071f1c0380a3502c427aba98a2fb5edf2f7cfdeea1676c',
    model_kwargs = {"temperature": 0.95,"top_p": 1, "max_new_tokens":512}
)

current_model = phi
current_model('Hello. How are you?')

'Hello. How are you? (abeded (abed (abed (abed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed (bed ('

In [14]:
task = tasks['15']
prompt_prefix = 'You are an expert Python programmer, and here is your task: {} Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:\n'

print(prompt_prefix.format(task['prompt']) + task['test'])

You are an expert Python programmer, and here is your task: Write a function to split a string at lowercase letters. Do not output any clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert split_lowerstring("AbCd")==['bC','d']
assert split_lowerstring("Python")==['y', 't', 'h', 'o', 'n']
assert split_lowerstring("Programming")==['r', 'o', 'g', 'r', 'a', 'm', 'm', 'i', 'n', 'g']


In [12]:
#model_nickname = 'llama3_8B'
#model_nickname = 'mistral_7B'
#model_nickname = 'gemma_7B'
#model_nickname = 'phixtral_8B'
#model_nickname = 'gemma_2B'
#model_nickname = 'solar_10B'
#model_nickname = 'phixtral'
#model_nickname = 'qwen_1_5_7B'
#model_nickname = 'mamba2'
#model_nickname = 'yi_6b'
model_nickname = 'phi'
time_stamp     = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-2]
file_name      = f'logs/{model_nickname}_samples_{time_stamp}.jsonl'
file_name

'logs/phi_samples_20250105_023208_8899.jsonl'

In [15]:
prompt_prefix = 'You are an expert Python programmer, and here is your task: {} Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:\n'

num_samples_per_task = 1
completions = []

for task_id, task_body in tasks.items():
    print(task_id)
    #if task_id == '15':
    #        break
    for i in range(num_samples_per_task):
        start_time  = time.time()
        full_prompt = prompt_prefix.format(task_body['prompt']) + task_body['test']
        try:
            completion  = current_model( full_prompt )
        except Exception as e:
            completion = f"Error generating completion:\n{e}"
        
        completion = remove_triple_backticks_at_edges(completion.strip())
        completions.append( {'task_id': task_id, 'completion': completion} )
        write_jsonl(file_name, completions)
        print('Prompt:\n', full_prompt, '\n', '-'*75, '\n', sep='')
        print('Completion:\n', completion, sep='')
        print(f"\nTime elapsed: {(time.time() - start_time):.4f} seconds\n", '\n', '='*100, '\n', sep='')

11
Prompt:
You are an expert Python programmer, and here is your task: Write a python function to remove first and last occurrence of a given character from the string. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H"
---------------------------------------------------------------------------

Completion:
You are an expert Python programmer, and here is your task: Write a python function to remove first and last occurrence of a given character from the string. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H" to, not, not, not, not, not, not, not, in, in, in, in, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, 

Prompt:
You are an expert Python programmer, and here is your task: Write a function to remove characters from the first string which are present in the second string. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert remove_dirty_chars("probasscurve", "pros") == 'bacuve'
assert remove_dirty_chars("digitalindia", "talent") == 'digiidi'
assert remove_dirty_chars("exoticmiles", "toxic") == 'emles' 
---------------------------------------------------------------------------

Completion:
You are an expert Python programmer, and here is your task: Write a function to remove characters from the first string which are present in the second string. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert remove_dirty_chars("probasscurve", "pros") == 'bacuve'
assert remove_dirty_chars("digitalindia", "talent") == 'digiidi'
assert remove_dirty_chars("

Prompt:
You are an expert Python programmer, and here is your task: Write a python function to find the product of non-repeated elements in a given array. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert find_Product([1,1,2,3],4) == 6
assert find_Product([1,2,3,1,1],5) == 6
assert find_Product([1,1,4,5,6],5) == 120
---------------------------------------------------------------------------

Completion:
You are an expert Python programmer, and here is your task: Write a python function to find the product of non-repeated elements in a given array. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert find_Product([1,1,2,3],4) == 6
assert find_Product([1,2,3,1,1],5) == 6
assert find_Product([1,1,4,5,6],5) == 120, in, in, in, a, in, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a

Prompt:
You are an expert Python programmer, and here is your task: Write a python function to find the largest prime factor of a given number. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert max_Prime_Factors(15) == 5
assert max_Prime_Factors(6) == 3
assert max_Prime_Factors(2) == 2
---------------------------------------------------------------------------

Completion:
You are an expert Python programmer, and here is your task: Write a python function to find the largest prime factor of a given number. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert max_Prime_Factors(15) == 5
assert max_Prime_Factors(6) == 3
assert max_Prime_Factors(2) == 2, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a, a

Time elapsed: 6.4818 seconds

Prompt:
You are an expert Python programmer, and here is your task: Write a function to find frequency of the elements in a given list of lists using collections module. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert freq_element([[1, 2, 3, 2], [4, 5, 6, 2], [7, 1, 9, 5]])==({2: 3, 1: 2, 5: 2, 3: 1, 4: 1, 6: 1, 7: 1, 9: 1})
assert freq_element([[1,2,3,4],[5,6,7,8],[9,10,11,12]])==({1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1})
assert freq_element([[15,20,30,40],[80,90,100,110],[30,30,80,90]])==({30: 3, 80: 2, 90: 2, 15: 1, 20: 1, 40: 1, 100: 1, 110: 1})
---------------------------------------------------------------------------

Completion:
You are an expert Python programmer, and here is your task: Write a function to find frequency of the elements in a given list of lists using collections module. Do not output clarifications. Output only runnable Python code and nothing else

...

Prompt:
You are an expert Python programmer, and here is your task: Write a python function to find the first natural number whose factorial is divisible by x. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert first_Factorial_Divisible_Number(10) == 5
assert first_Factorial_Divisible_Number(15) == 5
assert first_Factorial_Divisible_Number(5) == 4
---------------------------------------------------------------------------

Completion:
You are an expert Python programmer, and here is your task: Write a python function to find the first natural number whose factorial is divisible by x. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert first_Factorial_Divisible_Number(10) == 5
assert first_Factorial_Divisible_Number(15) == 5
assert first_Factorial_Divisible_Number(5) == 4, but, but, but, but, but, but, but, but, but, but, but, but, but, b

Prompt:
You are an expert Python programmer, and here is your task: Write a python function to find the frequency of the largest value in a given array. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert frequency_Of_Largest(5,[1,2,3,4,4]) == 2
assert frequency_Of_Largest(3,[5,6,5]) == 1
assert frequency_Of_Largest(4,[2,7,7,7]) == 3
---------------------------------------------------------------------------

Completion:
You are an expert Python programmer, and here is your task: Write a python function to find the frequency of the largest value in a given array. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert frequency_Of_Largest(5,[1,2,3,4,4]) == 2
assert frequency_Of_Largest(3,[5,6,5]) == 1
assert frequency_Of_Largest(4,[2,7,7,7]) == 3, and when we have been a high, and when we have been a high, and when we have been a high, in the

Prompt:
You are an expert Python programmer, and here is your task: Write a function to convert the given binary tuple to integer. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert binary_to_integer((1, 1, 0, 1, 0, 0, 1)) == '105'
assert binary_to_integer((0, 1, 1, 0, 0, 1, 0, 1)) == '101'
assert binary_to_integer((1, 1, 0, 1, 0, 1)) == '53'
---------------------------------------------------------------------------

Completion:
You are an expert Python programmer, and here is your task: Write a function to convert the given binary tuple to integer. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert binary_to_integer((1, 1, 0, 1, 0, 0, 1)) == '105'
assert binary_to_integer((0, 1, 1, 0, 0, 1, 0, 1)) == '101'
assert binary_to_integer((1, 1, 0, 1, 0, 1)) == '53' is a high, but, but, but, but, but, but, but, but, but, but, but, but, but, b

Prompt:
You are an expert Python programmer, and here is your task: Write a python function to find common divisor between two numbers in a given pair. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert num_comm_div(2,4) == 2
assert num_comm_div(2,8) == 2
assert num_comm_div(12,24) == 6
---------------------------------------------------------------------------

Completion:
You are an expert Python programmer, and here is your task: Write a python function to find common divisor between two numbers in a given pair. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert num_comm_div(2,4) == 2
assert num_comm_div(2,8) == 2
assert num_comm_div(12,24) == 6, and when we have some, and when we have some, and when we have some, and when we have some, and when we have some, and when we have some, and when we have, and when we have, and when we shou

Prompt:
You are an expert Python programmer, and here is your task: Write a function to check if the common elements between two given lists are in the same order or not. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert same_order(["red","green","black","orange"],["red","pink","green","white","black"])==True
assert same_order(["red","pink","green","white","black"],["white","orange","pink","black"])==False
assert same_order(["red","green","black","orange"],["red","pink","green","white","black"])==True
---------------------------------------------------------------------------

Completion:
You are an expert Python programmer, and here is your task: Write a function to check if the common elements between two given lists are in the same order or not. Do not output clarifications. Output only runnable Python code and nothing else. Your code should satisfy these tests:

assert same_order(["red","green","black","or

In [4]:
# if the above cell has not been run
#file_name = 'logs/samples_20241127_155444_8511.jsonl'

In [16]:
evaluate_functional_correctness(file_name, k=[1], mode='mbpp')

Reading samples...


500it [00:00, 11442.09it/s]


Running test suites...


100%|██████████████████████████████████████████████████████| 500/500 [00:32<00:00, 15.47it/s]


Writing results to logs/phi_samples_20250105_023208_8899.jsonl_results.jsonl...


100%|███████████████████████████████████████████████████| 500/500 [00:00<00:00, 69108.02it/s]


{'pass@1': 0.0}

# Appendix

## Models Tried, but Found Not Useful

In [10]:
mistral_7b = Replicate(
    model        = 'mistralai/mistral-7b-v0.1:3e8a0fb6d7812ce30701ba597e5080689bef8a013e5c6a724fafb108cc2426a0',
    model_kwargs = {"temperature": 0.95,"top_p": 1, "max_new_tokens":1000}
)

gemma_7b = Replicate(
    model        = 'google-deepmind/gemma-7b-it:2790a695e5dcae15506138cc4718d1106d0d475e6dca4b1d43f42414647993d5',
    model_kwargs = {"temperature": 0.95,"top_p": 1, "max_new_tokens":512}
)

gemma_2b = Replicate(
    model        = 'google-deepmind/gemma-2b-it:dff94eaf770e1fc211e425a50b51baa8e4cac6c39ef074681f9e39d778773626',
    model_kwargs = {"temperature": 0.95,"top_p": 1, "max_new_tokens":512}

)

gptj_6b = Replicate(
    model = 'replicate/gpt-j-6b:b3546aeec6c9891f0dd9929c2d3bedbf013c12e02e7dd0346af09c37e008c827',
    model_kwargs = {"temperature": 0.95,"top_p": 1, "max_new_tokens":512}
)

yi_6b = Replicate(
    model        = '01-ai/yi-6b:d302e64fad6b4d85d47b3d1ed569b06107504f5717ee1ec12136987bec1e94f1',
    model_kwargs = {"temperature": 0.95, "top_p": 1, "max_new_tokens":512}
)

phi = Replicate(
    model        = 'lucataco/phi-2:740618b0c24c0ea4ce5f49fcfef02fcd0bdd6a9f1b0c5e7c02ad78e9b3b190a6',
    model_kwargs = {"temperature": 0.95,"top_p": 1, "max_new_tokens":512}
)

## Using Replicate Without LangChain

__This approach works too, but the ouput is a list of strings. Joining over a space introduces indentation issues. Joining on an empty string seems to be better. But is there a risk of not joining correctly?__

In [61]:
import replicate

model         = "meta/meta-llama-3-8b-instruct"
model2        = "meta/meta-llama-3-8b"
prompt_prefix = '''
Act as an experienced Python software developer. Complete the following starter code.
Concatenate the starter code and the completion. Do not modify the starter code when you do this.
Double check and make sure that the indentation and sytax in the output code comply with the Python language rules and that the code is runnable.
Do not output irrelevant human phrases like: "Here is the complete code.", triple backticks or any other irrelevant symbols before or after the code.
Output only the unmodified starter code + its completion and nothing else:\n'
'''
client        = replicate.Client()
completions   = []

for task_id, task_body in tasks.items():
    if task_id == 'HumanEval/15':
        break
    start_time = time.time()
    input_params = {
        "prompt":         prompt_prefix + task_body["prompt"],
        "temperature":    0.5,
        "top_p":          1.0,
        "max_new_tokens": 1000,
    }
    output = client.run(model, input=input_params)
    print(''.join(output))
    completions.append(output)
    print(f"\nTime elapsed: {(time.time() - start_time):.4f} seconds\n")
    print('\n', '='*100, '\n')

2024-11-15 18:45:16,954 | INFO | HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    numbers.sort()
    for i in range(len(numbers) - 1):
        if abs(numbers[i] - numbers[i + 1]) <= threshold:
            return True
    return False

Time elapsed: 1.2501 seconds



2024-11-15 18:45:18,914 | INFO | HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"


```
from typing import List


def separate_paren_groups(paren_string: str) -> List[str]:
    result = []
    stack = []
    temp = ""
    for char in paren_string:
        if char == " ":
            continue
        if char == "(":
            stack.append(char)
            temp += char
        elif char == ")":
            stack.pop()
            temp += char
            if not stack:
                result.append(temp)
   

In [64]:
for idx, c in enumerate(completions):
    print(idx, '\n', ''.join(c))
    print('\n', '='*100, '\n')

0 
 

def has_close_elements(numbers: List[float], threshold: float) -> bool:
    numbers.sort()
    for i in range(len(numbers) - 1):
        if abs(numbers[i] - numbers[i + 1]) <= threshold:
            return True
    return False


1 
 

```
from typing import List


def separate_paren_groups(paren_string: str) -> List[str]:
    result = []
    stack = []
    temp = ""
    for char in paren_string:
        if char == " ":
            continue
        if char == "(":
            stack.append(char)
            temp += char
        elif char == ")":
            stack.pop()
            temp += char
            if not stack:
                result.append(temp)
                temp = ""
    return result
```


2 
 

def truncate_number(number: float) -> float:
    """ Given a positive floating point number, it can be decomposed into
    and integer part (largest integer smaller than given number) and decimals
    (leftover part always smaller than 1).

    Return the decimal part of the 

In [55]:
for task_id, task_body in tasks.items():
    if task_id == 'HumanEval/15':
        break
    print(task_id)
    print(task_body['prompt'])
    print('\n', '='*100, '\n')

HumanEval/0
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



HumanEval/1
from typing import List


def separate_paren_groups(paren_string: str) -> List[str]:
    """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to
    separate those group into separate strings and return the list of those.
    Separate groups are balanced (each open brace is properly closed) and not nested within each other
    Ignore any spaces in the input string.
    >>> separate_paren_groups('( ) (( )) (( )( ))')
    ['()', '(())', '(()())']
    """



HumanEval/2


def truncate_number(number: float) -> float:
    """ Given a positive floating point number, it can be

## How To Use Logging

In [2]:
# in case I need logging
# ATTENTION: introduces additional logging when calling Replicate models which slows down the process
logger = logging.getLogger()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')

stdout_handler = logging.StreamHandler(sys.stdout)
stdout_handler.setLevel(logging.DEBUG)
stdout_handler.setFormatter(formatter)

file_handler = logging.FileHandler('logs/logs.log')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)


logger.addHandler(file_handler)
logger.addHandler(stdout_handler)

In [3]:
logger.debug('This is a debig message.')
logger.warning('This is a warning message')
logger.info('This is a log message!')
logger.error('This is an error message.')
logger.critical('This is a critical message')
# Shut down the logger
#logging.shutdown()

2024-11-15 19:27:49,593 | WARNING | This is a warning message
2024-11-15 19:27:49,595 | INFO | This is a log message!
2024-11-15 19:27:49,597 | ERROR | This is an error message.
2024-11-15 19:27:49,598 | CRITICAL | This is a critical message


In [69]:
completions2 = []
for d in completions:
    completions2.append( {'task_id': d['task_id'], 'completion': remove_last_triple_backticks(d['completion'])} )

In [71]:
file_name = 'logs/qwen_1_5_7B_samples_20250103_035607_8043_last_triple_backticks_removed.jsonl'

In [74]:
write_jsonl(file_name, completions2)